# week 3 hw

In [1]:
# imports
!pip install statsmodels
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import statsmodels.api as sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 32.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [statsmodels] [statsmodels]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
hw3_df = pd.read_csv('homework_3.1.csv')
hw3_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  100 non-null    int64  
 1   time        100 non-null    int64  
 2   value1      100 non-null    float64
 3   value2      100 non-null    float64
 4   value3      100 non-null    float64
dtypes: float64(3), int64(2)
memory usage: 4.0 KB


In [6]:
hw3_df[:60]

,Unnamed: 0,time,value1,value2,value3
0,0,0,1.764052,1.883151,-0.369182
1,1,1,0.420157,-1.327759,-0.219379
2,2,2,1.018738,-1.230485,1.139660
3,3,3,2.300893,1.029397,0.715264
4,4,4,1.947558,-1.093123,0.720132
5,5,5,-0.877278,2.043621,-1.516956
6,6,6,1.070088,-0.293619,0.095674
7,7,7,-0.011357,-0.607455,-0.598031
8,8,8,0.056781,2.082942,0.439925
9,9,9,0.590599,1.660515,0.081850


In [7]:
# Q1 Which dataset is most likely to have a discontinuity (or has the strongest discontinuity) in the value at the event (at time = 50)? 


outcomes = ["value1", "value2", "value3"]
cutoff = 50

results = []

for outcome in outcomes:
    df = hw3_df.copy()
    df["post"] = (df["time"] >= cutoff).astype(int)
    df["time_c"] = df["time"] - cutoff
    
    y = df[outcome]
    X = sm.add_constant(df[["time_c", "post"]])
    model = sm.OLS(y, X).fit(cov_type="HC1")
    
    coef = model.params["post"]
    pval = model.pvalues["post"]
    
    results.append({
        "outcome": outcome,
        "jump_coef": coef,
        "p_value": pval,
        "R2": model.rsquared
    })

import pandas as pd
res_df = pd.DataFrame(results).sort_values("p_value")
print(res_df)


  outcome  jump_coef   p_value        R2
2  value3   1.767254  0.000016  0.777273
0  value1   0.850813  0.108956  0.651257
1  value2   0.682746  0.128447  0.539919


In [8]:
# Q2

outcomes = ["value1", "value2", "value3"]
cutoff = 50

results_slope = []

for outcome in outcomes:
    df = hw3_df.copy()
    df["post"] = (df["time"] >= cutoff).astype(int)
    df["time_c"] = df["time"] - cutoff
    df["time_c_post"] = df["time_c"] * df["post"]
    
    y = df[outcome]
    X = sm.add_constant(df[["time_c", "post", "time_c_post"]])
    model = sm.OLS(y, X).fit(cov_type="HC1")
    
    coef = model.params["time_c_post"]
    pval = model.pvalues["time_c_post"]
    
    results_slope.append({
        "outcome": outcome,
        "slope_change_coef": coef,
        "p_value": pval,
        "R2": model.rsquared
    })

res_slope_df = pd.DataFrame(results_slope).sort_values("p_value")
print(res_slope_df)


  outcome  slope_change_coef       p_value        R2
0  value1           0.105325  9.709709e-18  0.789732
2  value3           0.050695  3.471887e-05  0.807159
1  value2           0.036921  2.254570e-02  0.569465


In [9]:
hw3a_df = pd.read_csv('homework_3.2.a.csv')
hw3b_df = pd.read_csv('homework_3.2.b.csv')

In [10]:
hw3a_df[:60]

,Unnamed: 0,group1,time1,outcome1
0,0,0,0,0.882026
1,1,0,1,1.600079
2,2,0,0,0.489369
3,3,0,1,2.520447
4,4,0,0,0.933779
5,5,0,1,0.911361
6,6,0,0,0.475044
7,7,0,1,1.324321
8,8,0,0,-0.051609
9,9,0,1,1.605299


In [11]:
hw3b_df[:60]

,Unnamed: 0,group2,time2,outcome2
0,0,0,0,0.667155
1,1,0,1,2.470969
2,2,0,0,-0.506778
3,3,0,1,1.525657
4,4,0,0,0.273664
5,5,0,1,1.641776
6,6,0,0,0.648928
7,7,0,1,-0.781693
8,8,0,0,-0.059189
9,9,0,1,1.686840


In [12]:
# DID for Q3-5

def run_did(df, group_col, time_col, outcome_col):
    # Create interaction term
    df["did"] = df[group_col] * df[time_col]
    
    y = df[outcome_col]
    X = sm.add_constant(df[[group_col, time_col, "did"]])
    
    model = sm.OLS(y, X).fit(cov_type="HC1")  # robust SEs
    return model

# Run DiD for dataset A
model_a = run_did(hw3a_df.copy(), "group1", "time1", "outcome1")
print("=== Dataset A (hw3a_df) ===")
print(model_a.summary())
print("\nDiD estimate (treatment effect) =", model_a.params["did"])

# Run DiD for dataset B
model_b = run_did(hw3b_df.copy(), "group2", "time2", "outcome2")
print("\n=== Dataset B (hw3b_df) ===")
print(model_b.summary())
print("\nDiD estimate (treatment effect) =", model_b.params["did"])

# Compare results
did_a = model_a.params["did"]
pval_a = model_a.pvalues["did"]
did_b = model_b.params["did"]
pval_b = model_b.pvalues["did"]

print("\n--- Comparison ---")
print(f"Dataset A: DiD = {did_a:.4f}, p = {pval_a:.4g}")
print(f"Dataset B: DiD = {did_b:.4f}, p = {pval_b:.4g}")

if abs(did_a) > abs(did_b):
    print("=> Dataset A likely has the larger treatment effect.")
else:
    print("=> Dataset B likely has the larger treatment effect.")


=== Dataset A (hw3a_df) ===
                            OLS Regression Results                            
Dep. Variable:               outcome1   R-squared:                       0.899
Model:                            OLS   Adj. R-squared:                  0.899
Method:                 Least Squares   F-statistic:                     2807.
Date:                Tue, 16 Sep 2025   Prob (F-statistic):               0.00
Time:                        21:37:21   Log-Likelihood:                -712.28
No. Observations:                1000   AIC:                             1433.
Df Residuals:                     996   BIC:                             1452.
Df Model:                           3                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0258   